# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
import torch
from IPython.display import clear_output
import time
import pygame 

pygame 2.5.2 (SDL 2.28.2, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#| export
class Car:
    def __init__(self):
        self.location = [0, 4]  # start location
        self.degree = 0  # 0 is default | 360 loops back to 0 (since we're using degrees)
        self.original_sprite = pygame.image.load('car.png')  # Load your sprite image here
        self.sprite = self.original_sprite

    # Rotation
    def rotate_clockwise(self):
        self.degree =(self.degree + (360/4)) % 360 # 360/4 makes a perfect turn each time, reducing difficulty
        # (self.degree + (360/8)) % 360 ## 360/80 is the maximum freedom you can have since there are 8 blocks to move to
        self.rotate_sprite()

    def rotate_counter_clockwise(self):
        self.degree = (self.degree - (360/4)) % 360 # 360/2 makes a perfect turn each time, reducing difficulty
        # (self.degree - (360/8)) % 360 ## 360/80 is the maximum freedom you can have since there are 8 blocks to move to
        self.rotate_sprite()

    def rotate_sprite(self):
        self.sprite = pygame.transform.rotate(self.original_sprite, self.degree)
        
    # Movement
    def move_down(self):
        self.location[0] += 1

    def move_up(self):
        self.location[0] -= 1

    def move_right(self):
        self.location[1] += 1

    def move_left(self):
        self.location[1] -= 1

    def round_to_closest(self, value, options):
        return min(options, key=lambda x: abs(value - x))

    def move_forward(self):
        options = [0, 360, 90, 180, 270, 
                   ((0 + 90)/2), ((90 + 180)/2), ((180 + 270)/2), ((270 + 360)/2)]
        closest = self.round_to_closest(self.degree, options)
        # Horizontal & Vertical
        if closest == 0:
            self.move_right()
            print('moved right')
        elif closest == 360:
            self.move_right()
            print('moved right')
        elif closest== 90:
            self.move_up()
            print('moved up')
        elif closest == 180:
            self.move_left()
            print('moved left')
        elif closest == 270:
            self.move_down() 
            print('moved down')
        # Diagonals
        elif closest == (0 + 90)/2:
            self.move_right()
            self.move_up()
        elif closest == (90 + 180)/2:
            self.move_up()
            self.move_left()
        elif closest == (180 + 270)/2:
            self.move_left()
            self.move_down()
        elif closest == (270 + 360)/2:
            self.move_down()
            self.move_right()

In [5]:
#| export
class Game:
    def __init__(self, map_height, map_width):
        self.driving_plane = torch.zeros(map_height, map_width, dtype=torch.float16) # area "above" the road that the car sits on
        self.map = torch.zeros(map_height, map_width, dtype=torch.float16) # the physical location of the road 
        self.map_one()
        self.car = Car()
        self.objective = [0, 3] # behind car in default version of map
        self.crash = 0
        self.spawn_objective()
        self.TUI = True
        self.GUI = False


    def map_one(self):
        self.map[0][0] = 3
        self.map[0][1:7] = 5
        self.map[0][7] = 1
        self.map[1:8, 7]= 6
        self.map[8][7] = 2
        self.map[8][1:7] = 5
        self.map[8][0] = 4
        self.map[1:8, 0]= 6

        
    def step(self):
        try:
            self.despawn_car()# remove car
            self.car.move_forward()
            self.spawn_car()
            if self.map[self.car.location[0], self.car.location[1]] == 0:
                self.crash = 1
                print('off road')
        except IndexError:
            print('out of bounds')
            self.crash = 1 
        if self.TUI == True: self.tui()  

    def tui(self):
        print(f'Reward: {self.reward_condition()} | Crash: {self.crash}')
        print(self.driving_plane)
        clear_output(wait=True)     

    # Environment Stuff
    def setup(self): 
        return 
    def reward_condition(self): return 1 if self.car.location == self.objective else -1
    # # Car Stuff
    def spawn_car(self): self.driving_plane[self.car.location[0]][self.car.location[1]] = self.car.degree # spawn car
    def despawn_car(self): self.driving_plane[self.car.location[0]][self.car.location[1]] = 0
    # Objective Stuff
    def spawn_objective(self): self.driving_plane[self.objective[0], self.objective[1]] = 2
    def despawn_objective(self): self.driving_plane[self.objective[0], self.objective[1]] = 0    

In [6]:
#| export
road_sprite_map = {
    1: pygame.image.load('../map/PNGs/curve 1.png'),
    2: pygame.image.load('../map/PNGs/curve 2.png'),
    3: pygame.image.load('../map/PNGs/curve 3.png'),
    4: pygame.image.load('../map/PNGs/curve 4.png'),
    5: pygame.image.load('../map/PNGs/straight 1.png'),
    6: pygame.image.load('../map/PNGs/straight 2.png'),
}

In [24]:
#| export
game = Game(10, 10)
game.TUI = True
grid = False
BLACK = (0, 0, 0)
WHITE = (200, 200, 200)
WINDOW_HEIGHT = 1000
WINDOW_WIDTH = 1000


def draw_grid():
    blockSize = int(WINDOW_HEIGHT/len(game.driving_plane))  # Set the size of the grid block
    for x in range(0, WINDOW_WIDTH, blockSize):
        for y in range(0, WINDOW_HEIGHT, blockSize):
            rect = pygame.Rect(x, y, blockSize, blockSize)
            pygame.draw.rect(SCREEN, 'dark green', rect)
    
    # Map STUFF
    for i, row in enumerate(game.map):
        for j, val in enumerate(row):
            x = j * blockSize
            y = i * blockSize
            
            # Objective STUFF
            objective_gui_x = game.objective[1] * blockSize
            objective_gui_y = game.objective[0] * blockSize
            if (x, y) == (objective_gui_x, objective_gui_y):
                pygame.draw.rect(SCREEN, 'white', pygame.Rect(x, y, blockSize, blockSize))
            elif val != 0:
                road_sprite = road_sprite_map[int(val)]
                # Calculate the scale factor while maintaining aspect ratio
                sprite_width, sprite_height = road_sprite.get_size()
                scale_factor = min(blockSize / sprite_width, blockSize / sprite_height) 
                new_size = (int(sprite_width * scale_factor), int(sprite_height * scale_factor))
                scaled_sprite = pygame.transform.scale(road_sprite, new_size)
                # Calculate the position based on the grid block size and indices
                SCREEN.blit(scaled_sprite, (x, y))
            
            if grid == True:
                pygame.draw.rect(SCREEN, BLACK, pygame.Rect(x, y, blockSize, blockSize), 1)
            
            # Car STUFF
            car_gui_x = game.car.location[1] * blockSize
            car_gui_y = game.car.location[0] * blockSize
            car_sprite = game.car.sprite
            sprite_width, sprite_height = car_sprite.get_size()
            # Calculate the scale factor for the car sprite
            car_scale_factor = min(blockSize / sprite_width, blockSize / sprite_height) 
            new_size = (int(sprite_width * car_scale_factor), int(sprite_height * car_scale_factor))
            scaled_sprite = pygame.transform.scale(car_sprite, new_size)
            # Calculate offsets to center the scaled sprite within the grid cell
            offset_x = (blockSize - new_size[0]) // 2
            offset_y = (blockSize - new_size[1]) // 2
            SCREEN.blit(scaled_sprite, (car_gui_x + offset_x, car_gui_y + offset_y))
def main():
    global SCREEN, CLOCK
    pygame.init()
    SCREEN = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))#,pygame.FULLSCREEN)
    CLOCK = pygame.time.Clock()
    SCREEN.fill(WHITE)
    game.car.sprite = pygame.image.load('car.png') # .convert() is supposed to speed things up but doens't work for me        
    fps = 5 # AKA car speed 
    while True: # Game loop
        if game.crash == 1:
            break
        game.step()
        draw_grid()
        for event in pygame.event.get():
            if event.type == pygame.QUIT or game.crash == 1:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                # if event.key == pygame.K_UP:
                #     game.car.move_forward() 
                if event.key == pygame.K_LEFT:
                    game.car.rotate_counter_clockwise()
                elif event.key == pygame.K_RIGHT:
                    game.car.rotate_clockwise()
        pygame.display.update()
        CLOCK.tick(fps) 
        if game.reward_condition() == 1: 
            print('victory') 
            break 

In [25]:
#| export
main()

victory


In [ ]:
# # TODO
# - [x] Pygame UI
# - [x] Rewrite game functions to be stateless 
# - [x] Unify Backend and Pygame
    # - [x] initial state
    # - [x] step update
    # - [x] rotation
    # - [x] Car should be sprite
    # - [x] Rotate sprite
    # - [x] change to forward

----
    # Track
    # - [x] Tileset (one in download)
    # - [x] add to PyTorch tensor
    # - [x] Functionality moved to PyTorch and visualized in PyGame
    # - [x] out of bounds for hitting track edge (trying to go into tile otherside of track
----
    # AI 
    # [ ] 0 degree for car can't be default (since you can't distinguish from empty cell)
    # [ ] Q learning 
    # [ ] Deep Q 
    # [ ] Run many sims in parallel with GUI flag turned off 
    # [ ] test trained model with GUI on
    # AI difficulty tuning/progressive learning
    # [ ] Speed (higher for harder)
    # [ ] Turn sensitivity
        # - if turn sensitivity is high then we need less correct actions to turn a corner (although could be double edged sword when map is more complex than just single action 90 degree turn (like ours)

----
    # Small details
    # [x] car rescaling issue
    # [ ] make grid much bigger so turns don't seem so jerky
    # [ ] make state map into one 2d tensor instead of 2 1d tensors for running on GPU (test before and after)
    #Refactor
    # [ ] move car and map drawing to their own functions and call both in draw_grid()
    # [ ] car
    # [ ] map
----
# - Is our sim more efficient than NeuralNine?
    # - [ ] visualisation on?
    # - [ ] Visualisation off: 1 instance
    # - [ ] Visualisation off: Max number of instances we can spawn
----
